# Проект 8 Поиск лучшего региона для бурения скважин нефти

#### Описание проекта:

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [2]:
geodata0 = pd.read_csv('/datasets/geo_data_0.csv')
geodata1 = pd.read_csv('/datasets/geo_data_1.csv')
geodata2 = pd.read_csv('/datasets/geo_data_2.csv')

geodata0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [3]:
featuresgeo0 = geodata0.drop(['product','id'] , axis=1)
targetgeo0 = geodata0['product']

featuresgeo1 = geodata1.drop(['product','id'] , axis=1)
targetgeo1 = geodata1['product']

featuresgeo2 = geodata2.drop(['product','id'] , axis=1)
targetgeo2 = geodata2['product']

Предобработка данных не потребовалась, на взгляд данные корректные.

Из списка данных на обучение мы убрали колонку ID - по понятным причинам.

# 2. Обучение и проверка модели

# Разбиваем данные на обучающую и валидационную выборки в соотношении 75:25 для каждого региона:

In [4]:
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(featuresgeo0, targetgeo0, test_size=0.25, random_state=0)
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(featuresgeo1, targetgeo1, test_size=0.25, random_state=0)
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(featuresgeo2, targetgeo2, test_size=0.25, random_state=0)

# Модель для первого региона

In [5]:
modelgeo0 = LinearRegression()
modelgeo0.fit(features_train0, target_train0)
predictions0 = modelgeo0.predict(features_valid0)
rmse0 = (mean_squared_error(target_valid0, predictions0))**0.5

print('rmse = ',rmse0)
print('средний запас сырья =',geodata0['product'].mean())

rmse =  37.48100896950594
средний запас сырья = 92.50000000000001


# Модель для второго региона

In [6]:
modelgeo1 = LinearRegression()
modelgeo1.fit(features_train1, target_train1)
predictions1 = modelgeo1.predict(features_valid1)
rmse1 = (mean_squared_error(target_valid1, predictions1))**0.5

print('rmse = ',rmse1)
print('средний запас сырья =',geodata1['product'].mean())

rmse =  0.8872573052219321
средний запас сырья = 68.82500000000002


# Модель для третьего региона

In [7]:
modelgeo2 = LinearRegression()
modelgeo2.fit(features_train2, target_train2)
predictions2 = modelgeo2.predict(features_valid2)
rmse2 = (mean_squared_error(target_valid2, predictions2))**0.5

print('rmse = ',rmse2)
print('средний запас сырья =',geodata2['product'].mean())

rmse =  40.31290686044374
средний запас сырья = 95.00000000000004


Более высокий средний запас сырья коррелирует с более высоким rmse.

# 3. Подготовка к расчёту прибыли

In [8]:
research_in_region = 500

budget = 10000000000

well_cost = 50000000

profit_pet_barrel = 4500

# Не рассматривать регионы, в которых риск убытков выше 2.5%. Из оставшихся выбирается регион с наибольшей средней прибылью.

In [9]:
print('Минимальный объём месторождения в тыс баррелей для его рентабельной разработки',(well_cost//profit_pet_barrel)/1000)

Минимальный объём месторождения в тыс баррелей для его рентабельной разработки 11.111


In [10]:
print('Бюджета хватит на разработку',budget/well_cost,'месторождений')

Бюджета хватит на разработку 200.0 месторождений


# Функция для расчёта прибыли по набору отобранных месторождений и предсказаний модели

In [11]:
def add_prediction_profit(data,model):
    features = data.drop(['product','id'] , axis=1)
    prediction_product = model.predict(features)
    columns = ['prediction_product']
    prediction_data = pd.DataFrame(data = prediction_product,columns = columns)
    sum_prediction_product_top200region = prediction_data.sort_values(by = 'prediction_product', ascending = False)[:200].sum()[0]
    prediction_profit = sum_prediction_product_top200region*1000*profit_pet_barrel-budget
    result = prediction_profit
    return result

# 4. Расчёт прибыли и рисков 

In [13]:
state = np.random.RandomState(12345)
def spreading_profit(data,model):
    values = []
    for i in range(1000):
        subsample = data.sample(n=research_in_region, replace=True, random_state=state)
        values.append(add_prediction_profit(subsample,model))
    values = pd.Series(values)
    return values

In [14]:
values0 = spreading_profit(geodata0,modelgeo0)
values1 = spreading_profit(geodata1,modelgeo1)
values2 = spreading_profit(geodata2,modelgeo2)

In [15]:
def mean_and_interval(values,n):
    print('Для региона', n , 'средняя прибыль =',values.mean(),': 95%-й доверительный интервал =', st.t.interval(0.95, len(values)-1, values.mean(),values.sem()))

In [16]:
mean_and_interval(values0,0)
mean_and_interval(values1,1)
mean_and_interval(values2,2)

Для региона 0 средняя прибыль = 93450536518.77348 : 95%-й доверительный интервал = (93377136962.91835, 93523936074.62862)
Для региона 1 средняя прибыль = 94330889495.29306 : 95%-й доверительный интервал = (94208029727.7382, 94453749262.84792)
Для региона 2 средняя прибыль = 92747771784.90112 : 95%-й доверительный интервал = (92685351033.09055, 92810192536.7117)


# Риск убытков:

In [17]:
new_columns = ['prediction_profit']
data_values0 = pd.DataFrame(data = values0,columns = new_columns)
values0_sort = data_values0.sort_values(by = 'prediction_profit', ascending = True)
values0_sort

,prediction_profit
779,9.000522e+10
368,9.025250e+10
77,9.031562e+10
335,9.042509e+10
814,9.042967e+10
...,...
100,9.627857e+10
425,9.634718e+10
773,9.656530e+10
740,9.686761e+10


In [18]:
data_values1 = pd.DataFrame(data = values1,columns = new_columns)
values1_sort = data_values1.sort_values(by = 'prediction_profit', ascending = True)
values1_sort

,prediction_profit
216,8.842578e+10
481,8.880451e+10
863,8.886048e+10
583,8.920760e+10
806,8.932426e+10
...,...
30,9.988592e+10
91,1.000089e+11
988,1.001355e+11
250,1.003616e+11


In [19]:
data_values2 = pd.DataFrame(data = values2,columns = new_columns)
values2_sort = data_values2.sort_values(by = 'prediction_profit', ascending = True)
values2_sort

,prediction_profit
927,8.948354e+10
427,8.959538e+10
317,8.959667e+10
75,8.961845e+10
684,8.998544e+10
...,...
667,9.535574e+10
661,9.538497e+10
150,9.543571e+10
216,9.554801e+10


Вывод по риску: используя сортировку по возрастанию, найденно, что отрицательных значений - нет, значит и риска получить убыток - тоже.
    
Вывод по заданию: выгоднее всего выбрать регион "1", так как у него выше средняя прибыль, и 95% интервал доверительный находится - выше.